table of contents
1. Prepare
2. Data Import, Encoding
3. EDA
4. write df

In [2]:
import os, random, time

import numpy as np
import pandas as pd

# Prepare

## Define parameters

In [3]:
base_dir = os.path.join('..','..')
data_dir = os.path.join(base_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')

infile_list = os.listdir(processed_dir)
for i, infile in enumerate(infile_list):
    print(i, infile)

0 events.csv
1 matches
2 events
3 coaches.csv
4 .~lock.teams.csv#
5 teams.csv


## Data import

In [4]:
competition = 'Spain'
#df = pd.read_csv(os.path.join(processed_dir+'/events', 'events_'+competition+'.csv'))
df = pd.read_json(os.path.join(raw_dir+'/events', 'events_'+competition+'.json'))
team_df = pd.read_json(os.path.join(raw_dir, 'teams.json'))
player_df = pd.read_json(os.path.join(raw_dir, 'players.json'))


In [5]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422


In [6]:
df[['st_x', 'st_y', 'ed_x', 'ed_y']] = df.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']] if (len(xs['positions']) >= 2) else [xs['positions'][0]['x'], xs['positions'][0]['y'],None, None] , result_type='expand', axis=1)

In [8]:
team_df_indexed = team_df.set_index('wyId')

df = pd.merge(df, team_df_indexed, left_on='teamId', right_index=True)

In [10]:
player_df_indexed = player_df.set_index('wyId')

df = pd.merge(df, player_df_indexed, left_on='playerId', right_index=True)

In [11]:
#df.loc[df.eventName=='Shot'].head(30)
df.iloc[1890]

eventId                                                                  8
subEventName                                                   Simple pass
tags                                                        [{'id': 1801}]
playerId                                                              3542
positions                         [{'y': 70, 'x': 35}, {'y': 57, 'x': 54}]
matchId                                                            2565821
eventName                                                             Pass
teamId                                                                 682
matchPeriod                                                             1H
eventSec                                                           830.482
subEventId                                                              85
id                                                               233743391
st_x                                                                    35
st_y                     

In [72]:
df_pass = df.loc[df.eventName=='Pass'].copy()

In [73]:
df_pass.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,st_x,st_y,ed_x,ed_y
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419,37.0,61.0,50.0,50.0
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418,50.0,50.0,45.0,30.0
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420,45.0,30.0,38.0,12.0
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421,38.0,12.0,32.0,69.0
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422,32.0,69.0,31.0,37.0


In [74]:
#df_pass['st_x'] = None
#df_pass['st_y'] = None
#df_pass['ed_x'] = None
#df_pass['ed_y'] = None

#df_pass['list_positions'] = df_pass['positions'].apply(lambda x: list(x))

#df_pass['list_positions'].values
#df_pass[['st_x', 'st_y', 'ed_x', 'ed_y']] = df_pass.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']], result_type='expand', axis=1)
#df_pass['st_x'] = df_pass.apply(lambda xs: xs['positions'][0]['x'])


df_pass['success'] = df_pass.apply(lambda xs: 1 if 1801 in [tag['id'] for tag in xs['tags']] else 0, axis=1)

In [75]:
df_pass.head(20)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,st_x,st_y,ed_x,ed_y,success
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419,37.0,61.0,50.0,50.0,1
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418,50.0,50.0,45.0,30.0,1
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420,45.0,30.0,38.0,12.0,1
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421,38.0,12.0,32.0,69.0,1
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422,32.0,69.0,31.0,37.0,1
5,8,Simple pass,[{'id': 1801}],3277,"[{'y': 37, 'x': 31}, {'y': 83, 'x': 37}]",2565548,Pass,682,1H,18.480831,85,180864423,31.0,37.0,37.0,83.0,1
6,8,Simple pass,[{'id': 1801}],3695,"[{'y': 83, 'x': 37}, {'y': 53, 'x': 13}]",2565548,Pass,682,1H,23.978291,85,180864424,37.0,83.0,13.0,53.0,1
7,8,Simple pass,[{'id': 1801}],3486,"[{'y': 53, 'x': 13}, {'y': 41, 'x': 33}]",2565548,Pass,682,1H,28.831334,85,180864425,13.0,53.0,33.0,41.0,1
8,8,Simple pass,[{'id': 1801}],3277,"[{'y': 41, 'x': 33}, {'y': 79, 'x': 33}]",2565548,Pass,682,1H,30.844379,85,180864426,33.0,41.0,33.0,79.0,1
9,8,Simple pass,[{'id': 1801}],3695,"[{'y': 79, 'x': 33}, {'y': 98, 'x': 54}]",2565548,Pass,682,1H,34.267339,85,180864427,33.0,79.0,54.0,98.0,1


### Calculate sectors of the playing field

Divide the land into 18 rectangles, we have a grid of 6 x 3 that mark 18 zones (https://footballbh.net/2019/02/04/the-use-of-zone-14-the-key-to-winning-football-matches/)

Zones

* Horizontal zone 
    * left 
    * center 
    * right
* Vertical zone  
    * own field (low, middle, high) 
    * rival field (low, middle, high)


In [ ]:
sector           = ('own', 'rival')
vertical_zones   = (('left',25),('middle', 50),('right', 25))
horizontal_zones = (('low', 30), ('middle', 35), ('high', 35))


#for vz in vertical_zones:
    